<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import multiprocessing
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2024-10-30 21:35:49--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2024-10-30 21:35:49--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2024-10-30 21:35:50--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [ ]:
# Posibles bandas
os.listdir("./songs_dataset/")

['ludacris.txt',
 'prince.txt',
 'bieber.txt',
 'radiohead.txt',
 'johnny-cash.txt',
 'lady-gaga.txt',
 'bjork.txt',
 'leonard-cohen.txt',
 'disney.txt',
 'janisjoplin.txt',
 'bruce-springsteen.txt',
 'drake.txt',
 'dickinson.txt',
 'adele.txt',
 'r-kelly.txt',
 'cake.txt',
 'lil-wayne.txt',
 'rihanna.txt',
 'al-green.txt',
 'notorious-big.txt',
 'dj-khaled.txt',
 'lorde.txt',
 'Kanye_West.txt',
 'jimi-hendrix.txt',
 'Lil_Wayne.txt',
 'nursery_rhymes.txt',
 'beatles.txt',
 'nirvana.txt',
 'notorious_big.txt',
 'dolly-parton.txt',
 'alicia-keys.txt',
 'nickelback.txt',
 'eminem.txt',
 'kanye.txt',
 'kanye-west.txt',
 'bob-marley.txt',
 'lin-manuel-miranda.txt',
 'paul-simon.txt',
 'missy-elliott.txt',
 'nicki-minaj.txt',
 'joni-mitchell.txt',
 'michael-jackson.txt',
 'blink-182.txt',
 'britney-spears.txt',
 'patti-smith.txt',
 'bob-dylan.txt',
 'amy-winehouse.txt',
 'bruno-mars.txt',
 'dr-seuss.txt']

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)
df.head()

<ipython-input-4-91c4bad52ba1>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('songs_dataset/beatles.txt', sep='/n', header=None)


,0
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1846


### 1 - Preprocesamiento

In [ ]:
!pip install tensorflow

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:5]

[['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away'],
 ['now', 'it', 'looks', 'as', 'though', "they're", 'here', 'to', 'stay'],
 ['oh',
  'i',
  'believe',
  'in',
  'yesterday',
  'suddenly',
  "i'm",
  'not',
  'half',
  'the',
  'man',
  'i',
  'used',
  'to',
  'be'],
 ["there's", 'a', 'shadow', 'hanging', 'over', 'me'],
 ['oh',
  'yesterday',
  'came',
  'suddenly',
  'why',
  'she',
  'had',
  'to',
  'go',
  'i',
  "don't",
  'know',
  'she',
  "wouldn't",
  'say']]

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1846


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 445


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 113208.15625
Loss after epoch 1: 65953.953125
Loss after epoch 2: 65934.265625
Loss after epoch 3: 65711.96875
Loss after epoch 4: 63872.5
Loss after epoch 5: 64142.46875
Loss after epoch 6: 64058.8125
Loss after epoch 7: 64755.4375
Loss after epoch 8: 62585.5625
Loss after epoch 9: 60415.25
Loss after epoch 10: 59823.625
Loss after epoch 11: 58930.1875
Loss after epoch 12: 57726.4375
Loss after epoch 13: 56493.125
Loss after epoch 14: 55842.5625
Loss after epoch 15: 55862.5
Loss after epoch 16: 51695.8125
Loss after epoch 17: 49815.5
Loss after epoch 18: 49568.125
Loss after epoch 19: 48959.75


(156986, 287740)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('pretty', 0.9087147116661072),
 ('sleep', 0.8649678230285645),
 ('help', 0.8594305515289307),
 ('try', 0.8456019759178162),
 ('cry', 0.8387227654457092),
 ('little', 0.8273633718490601),
 ('not', 0.8181102871894836),
 ('seems', 0.8175203204154968),
 ('twist', 0.8154979348182678),
 ('peace', 0.812263011932373)]

In [ ]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('shake', -0.21786975860595703),
 ('our', -0.24495810270309448),
 ('come', -0.2613270878791809),
 ('on', -0.26722386479377747),
 ('bang', -0.27329546213150024),
 ('five', -0.2741836607456207),
 ('six', -0.2774038314819336),
 ('baby', -0.2776353061199188),
 ('work', -0.2790372967720032),
 ('four', -0.2792743146419525)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('five', 0.9786694049835205),
 ('three', 0.9761051535606384),
 ('six', 0.966891348361969),
 ('two', 0.9565720558166504),
 ('seven', 0.954232931137085),
 ('sixty', 0.9003373980522156),
 ('one', 0.816501259803772),
 ('us', 0.7762899398803711),
 ('crying', 0.772737979888916),
 ('strawberry', 0.7672452330589294)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[("can't", 0.9409592747688293),
 ('buy', 0.9399508237838745),
 ('much', 0.8996850848197937),
 ('hide', 0.8465201258659363),
 ('just', 0.8415144085884094)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: "Key 'diedaa' not present in vocabulary"

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689682483673096),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385538458824158),
 ('hope', 0.8361446261405945)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Dataset escogido: Spam email from Enron Dataset

##  1 - Descripcion del conjunto de datos  
El conjunto de datos a utilizar es el correspondiente a la base de datos “Enron Email
Dataset”. Contiene datos de alrededor de 150 usuarios con un total de alrededor de 500,000
mensajes de correo electronico. Originalmente el dataset se hizo publico por la Comision Federal
Regulatoria de Energıa de Estaos Unidos durante la investigacion alrededor del colapso de la
empresa Enron.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import multiprocessing
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PNL/clase_2/jupyter_notebooks/labeled_emails.csv")

In [ ]:
df.head()

,email,label
0,Subject: re : rankings\nthank you .,ham
1,"Subject: re : visit to enron\nvince ,\ndec . 2...",ham
2,Subject: research group move to the 19 th floo...,ham
3,Subject: christmas baskets\nthe christmas bask...,ham
4,"Subject: japan candidate\nvince ,\ni spoke wit...",ham


In [ ]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-20-23e21f4224e1>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [ ]:
sentence_tokens

Output hidden; open in https://colab.research.google.com to view.

## 2 - Instancia del modelo

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 5854


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 8184


## 3 - Entrenamiento

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 9135045.0
Loss after epoch 1: 7213221.0
Loss after epoch 2: 6578278.0
Loss after epoch 3: 6367690.0
Loss after epoch 4: 6102486.0
Loss after epoch 5: 5743756.0
Loss after epoch 6: 5605388.0
Loss after epoch 7: 5503004.0
Loss after epoch 8: 5413940.0
Loss after epoch 9: 5341404.0
Loss after epoch 10: 4457764.0
Loss after epoch 11: 1527968.0
Loss after epoch 12: 1479456.0
Loss after epoch 13: 1432784.0
Loss after epoch 14: 1381800.0
Loss after epoch 15: 1335640.0
Loss after epoch 16: 1293968.0
Loss after epoch 17: 1249512.0
Loss after epoch 18: 1210296.0
Loss after epoch 19: 1187480.0


(21591533, 28623680)

### 4 - Test

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["congratulations"], topn=5)
# promotion, card, purchasing, new

[('congrats', 0.6032657623291016),
 ('deserved', 0.49680399894714355),
 ('wolfin', 0.38669973611831665),
 ('merry', 0.37556132674217224),
 ('macmillan', 0.3686480224132538)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["promotion"], topn=5)

[('congrats', 0.4194207787513733),
 ('deserved', 0.3881804049015045),
 ('deserving', 0.3827337622642517),
 ('election', 0.38179779052734375),
 ('shannon', 0.358280211687088)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["cards"], topn=5)

[('debit', 0.5138387084007263),
 ('acts', 0.4018385112285614),
 ('depot', 0.3781992495059967),
 ('payroll', 0.3724546432495117),
 ('stored', 0.36612412333488464)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["purchasing"], topn=5)

[('uhc', 0.35434094071388245),
 ('lan', 0.3450513780117035),
 ('decrease', 0.34434276819229126),
 ('coulter', 0.33327993750572205),
 ('consolidated', 0.3308396637439728)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["new"], topn=5)

[('york', 0.40912848711013794),
 ('healthy', 0.3450511693954468),
 ('lap', 0.32198408246040344),
 ('vdg', 0.3171232044696808),
 ('fort', 0.31698963046073914)]

## Análisis de las palabras similares para cada ejemplo:

 ### Como es un dataset de correos Spam, se escogieron palabas que supongo pueden ser frecuentes en el cuerpo o asunto del correo:

1 -  "congratulations": 3/5
	('congrats', 0.6032657623291016) : correcto
 	('deserved', 0.49680399894714355): (Merecido) correcto
 	('wolfin', 0.38669973611831665)  : No reconozco la palabra
 	('merry', 0.37556132674217224)   : (Feliz) correcto
 	('macmillan', 0.3686480224132538 : No reconozco la palabra

2 -  "promotion": 4/5
	('congrats', 0.4194207787513733) : correcto
 	('deserved', 0.3881804049015045) : (merecido), puede que un contexto dado sea correcto
 	('deserving', 0.3827337622642517): (merecedor), puede que un contexto dado sea correcto
 	('election', 0.38179779052734375): (elección - elegido), puede que un contexto dado sea correcto
 	('shannon', 0.358280211687088)   : No reconozco la palabra

3 - "cards": 2/5
	('debit', 0.5138387084007263)    : correcto
 	('acts', 0.4018385112285614)     : incorrecto
 	('depot', 0.3781992495059967)    : correcto
 	('payroll', 0.3724546432495117)  : correcto
 	('stored', 0.36612412333488464)  : incorrecto

4 - "purchasing": 0/5
	('uhc', 0.35434094071388245)         : No reconozco la palabra, incorrecto
 	('lan', 0.3450513780117035)          : incorrecto
 	('decrease', 0.34434276819229126)    : incorrecto
 	('coulter', 0.33327993750572205)     : incorrecto
 	('consolidated', 0.3308396637439728) : incorrecto

5 - "new": 1/5
	('york', 0.40912848711013794)	     : Lo asocia a la ciudad, correcto
 	('healthy', 0.3450511693954468)      : incorrecto
 	('lap', 0.32198408246040344)         : incorrecto
 	('vdg', 0.3171232044696808)          : incorrecto
 	('fort', 0.31698963046073914)        : incorrecto

## Ahora grafico las palabras utilizando T-SNE.

La palabra buscada y las 5 mas similares, según el método utilizado

In [ ]:
def plot_selected_words(model, main_words):
    # Inicializar listas para almacenar datos de palabras, vectores y etiquetas de grupo
    all_words = []
    all_vectors = []
    all_labels = []

    for main_word in main_words:
        # Verificar si la palabra principal está en el vocabulario
        if main_word in model.wv.key_to_index:
            # Añadir la palabra principal
            all_words.append(main_word)
            all_vectors.append(model.wv[main_word])
            all_labels.append(main_word)  # Utilizar la palabra principal como grupo

            # Obtener palabras similares
            similar_words = [word for word, _ in model.wv.most_similar(main_word, topn=5)]
            for word in similar_words:
                if word in model.wv.key_to_index:
                    all_words.append(word)
                    all_vectors.append(model.wv[word])
                    all_labels.append(main_word)  # Etiqueta de grupo

    if not all_words:
        print("Ninguna de las palabras seleccionadas o sus similares existe en el vocabulario del modelo")
        return

    # Aplicar T-SNE
    vectors_2d = TSNE(n_components=2, random_state=0, perplexity=min(len(all_vectors) - 1, 30)).fit_transform(np.array(all_vectors))

    # Crear el gráfico
    fig = px.scatter(
        x=vectors_2d[:, 0],
        y=vectors_2d[:, 1],
        text=all_words,
        color=all_labels,  # Colorear por grupo de palabra principal
        title="Word Embeddings de palabras seleccionadas y sus similares"
    )

    # Modificar el diseño para mostrar las etiquetas
    fig.update_traces(textposition='top center')
    fig.show(renderer="colab")

# Lista de palabras principales que quieres visualizar
main_words = ["congratulations", "promotion", "cards", "purchasing", "new"]

# Graficar
plot_selected_words(w2v_model, main_words)

En la gráfica se ve que en general las palabras catalogadas como similares estan en misma región, aunque hay casos como el de la palabra "purchasing" que se nota que esta alejada de sus palabras similares.